In [2]:
from random import *
from decimal import *
from copy import *
from IPython.display import clear_output

# Constant
INFINITY = 1.0e400

class Player(object):
    
    HUMAN = 0
    ABPRUNE = 1
    
    def __init__(self, playerNum, playerType, ply=0):

        self.num = playerNum
        self.opp = 1 - playerNum
        self.type = playerType
        self.ply = ply

    def __repr__(self):

        return str(self.num)
        

    def score(self, board):

        if board.hasWon(self.num):
            return 100.0
        elif board.hasWon(self.opp):
            return 0.0
        else:
            return 50.0

    def alphaBetaMove(self, board, ply):

        move = -1
        alpha = -INFINITY
        beta = INFINITY
        score = -INFINITY
        turn = self
        for m in board.legalMoves(self):
            if ply == 0:
                return (self.score(board), m)
            if board.gameOver():
                return (-1, -1)  
            NB = deepcopy(board)
            NB.makeMove(self, m)
            opp = Player(self.opp, self.type, self.ply)
            s = opp.minABValue(nb, ply-1, turn, alpha, beta)
            if s > score:
                move = m
                score = s
            alpha = max(score, alpha)
        return score, move

    #lower bound
    def minABValue(self, board, ply, turn, alpha, beta):
        """ Find the alpha-beta value for the next move for this player
        at a given board configuation. Returns score."""
        if board.gameOver():
            return turn.score(board)
        score = INFINITY
        for m in board.legalMoves(self):
            if ply == 0:
                return turn.score(board)
            opponent = Player(self.opp, self.type, self.ply)
            NB = deepcopy(board)
            NB.makeMove(self, m)
            score = min(score, opponent.maxABValue(nextBoard, ply-1, turn, alpha, beta))
            if(score <= alpha):
                return score
            beta = min(beta, score)
        return score

    #higher bound
    def maxABValue(self, board, ply, turn, alpha, beta):
        """ Find the alpha-beta value for the next move for this player
            at a given board configuation. Returns score."""
        if board.gameOver():
            return turn.score(board)
        score = -INFINITY
        for m in board.legalMoves(self):
            if ply == 0:
                return turn.score(board)
            opponent = Player(self.opp, self.type, self.ply)
            NB = deepcopy(board)
            NB.makeMove(self, m)
            score = max(score, opponent.minABValue(nextBoard, ply-1, turn, alpha, beta))
            if (score >= beta):
                return score
            alpha = max(alpha, score)
        return score
     
    def space_check(board, position):
    
        return board[position] == ' '   
        
        
    def chooseMove(self, board):
        if self.type == self.HUMAN:
                move = input("Please enter your move:")
                while not board.legalMove(self, move):
                    print (move, "is not valid")
                    move = input( "Please enter your move:" )
                return move
        elif self.type == self.ABPRUNE:
            val, move = self.alphaBetaMove(board, self.ply)
            print ("chose move", move, " with value", val)
            return move           
        else:
            print ("Unknown player type")
            return -1

class TTTBoard(object):
    def __init__(self):
        self.SIZE = 3
        self.board = [' ']*(self.SIZE*self.SIZE)
        
    def space_check(self,board, position):
    
        return board[position] == ' '   
    
    def displayBoard(self):
        clear_output()
    
        print('   |   |')
        print(' ' + self.board[6] + ' | ' + self.board[7] + ' | ' + self.board[8])
        print('   |   |')
        print('-----------')
        print('   |   |')
        print(' ' + self.board[3] + ' | ' + self.board[4] + ' | ' + self.board[5])
        print('   |   |')
        print('-----------')
        print('   |   |')
        print(' ' + self.board[0] + ' | ' + self.board[1] + ' | ' + self.board[2])
        print('   |   |')

    def legalMove( self, player, move ):
        print(self.space_check(board,move))
        return move in [0,1,2,3,4,5,6,7,8] and self.space_check(board, move)

    def legalMoves( self, player ):
        moves = []
        for m in range( len(self.board)):
            if self.board[m] == ' ':
                moves += [m]
        return moves

    def makeMove( self, player, pos ):
        move = pos
        if player.num == 1:
            self.board[move] = 'X'
        else:
            self.board[move] = 'O'
        return True
    
    def rowWin( self, c ):
        for i in range(self.SIZE):
            if self.board[i*self.SIZE:(i+1)*self.SIZE] == [c]*self.SIZE:
                return True
        return False
    
    def colWin( self, c):
        for i in range(self.SIZE):
            col = []
            for j in range(self.SIZE):
                col += [self.board[j*self.SIZE+i]]
                if col == [c]*self.SIZE:
                    return True
        return False
    
    def diagWin( self, c ):
        diag = []
        offdiag = []
        for i in range(self.SIZE):
            diag += self.board[i*self.SIZE+i]
            offdiag += self.board[((i+1)*self.SIZE)-1-i]
            if diag == [c]*self.SIZE or offdiag == [c]*self.SIZE:
                return True
        return False
    
    def hasWonPlayer( self, c ):
        return self.rowWin(c) or self.colWin(c) or self.diagWin(c)
    
    def hasWon( self, playerNum ):
        if playerNum == 1:
            return self.hasWonPlayer( "X" )
        else:
            return self.hasWonPlayer( "O" )

    def gameOver(self):
        if self.hasWonPlayer("X") or self.hasWonPlayer("O"):
            return True
        else:
            for move in self.board:
                if move == ' ':
                    return False
            return True
        
    def reset( self ):
        self.board = [' ']*(self.SIZE*self.SIZE)
            

    def hostGame( self, player1, player2 ):
        #self.reset()
        turn = player1      # Keep track of whose turn it is
        wait = player2
        winner = 0
        rounds = 0
        while winner == 0 and rounds < self.SIZE*self.SIZE:
            self.displayBoard()
            pos = turn.chooseMove(self)
            self.makeMove( turn, pos )
            if self.hasWon(turn.num):
                winner = turn.num
            temp = turn
            turn = wait
            wait = temp
            rounds += 1
        self.displayBoard()
        if winner == 0:
            print ("Tie Game")
        else:
            if winner == 1:
                print ("X wins!")
            else:
                print ("O wins!")
        
def main():
    player1=Player(1, Player.ABPRUNE)
    player2=Player(2, Player.HUMAN)
    x = TTTBoard()
    x.hostGame(player1,player2)
    
if __name__=="__main__":
    main()

   |   |
   |   |  
   |   |
-----------
   |   |
   |   |  
   |   |
-----------
   |   |
 X |   |  
   |   |
Please enter your move:5


NameError: name 'board' is not defined